# PacBio run: Enhancer - BC coupling of CHEQ-seq Intron library
- PacBio run on CHEQ-seq Intron library
- Do enhancer=BC coupling specfically for this one
- Read structure: FW-primer + Chimeric Intron + Enhancer + Chimeric Intron + Barcode + REV-Primer

In [1]:
echo $PATH

/apps/leuven/skylake/2018a/software/Java/1.8.0_192:/apps/leuven/skylake/2018a/software/Java/1.8.0_192/bin:/apps/leuven/skylake/2018a/software/git/2.16.1-foss-2018a/bin:/apps/leuven/bin:/usr/local/bin:/usr/lpp/mmfs/bin:.:/usr/bin:/usr/sbin:/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/software/genius/anaconda3/2019.03/bin:/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/software/genius/git-flow/0.4.1/bin:/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/software/genius/tree/1.8.0/bin:/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/software/genius/nextflow/latest/bin:/staging/leuven/stg_00002/lcb/dwmax/software/genius/usr/bin:/apps/leuven/skylake/2018a/software/Perl/5.26.1-foss-2018a/bin:/apps/leuven/skylake/2018a/software/gettext/0.19.8.1-GCCcore-6.4.0-libxml2-2.9.7/bin:/apps/leuven/skylake/2018a/software/ncurses/6.0-GCCcore-6.4.0/bin:/apps/leuven/skylake/2018a/software/libxml2/2.9.7-GCCcore-6.4.0/bin:/apps/leuven/skylake/2018a/software/XZ/5.2.3-GCCcore-6.4.0/bin:/apps/leuven/skylake/2018a/softw

In [17]:
cd data/2917_DMauduit_PacBio

In [15]:
SAMPLE_NAME="intron_plasmid"

## Extract reads
- see 5pPlasmid Python notebook

## Quality checking before cleaning
- see 5pPlasmid Python notebook

## Demutliplex
- see 5pPlasmid Python notebook

## Intron Plasmid

### Separate forward from reverse reads

In [2]:
# # Use the 5' flanking sequence of the barcode
# _5p_flanking_seq_barcode="CACTCCCAGGTGGCGCGCCCTAG"
# rev_comp_5p_flanking_seq_barcode=$(echo "CACTCCCAGGTGGCGCGCCCTAG" | rev | tr "ACGT" "TGCA")

In [3]:
# # Use the 3' flanking sequence of the barcode
# _3p_flanking_seq_barcode="TGCAGGTCGCCTGCAGGCTTAAA"
# rev_comp_3p_flanking_seq_barcode=$(echo "TGCAGGTCGCCTGCAGGCTTAAA" | rev | tr "ACGT" "TGCA")

In [4]:
# echo $rev_comp_3p_flanking_seq_barcode

In [5]:
# cd data/2917_DMauduit_PacBio
# ls

In [6]:
# cutadapt \
#    -g forward=$_5p_flanking_seq_barcode \
#    -a reverse=$rev_comp_5p_flanking_seq_barcode \
#    --no-trim \
#    -o 03.demux/pacbio_3passes_intron_plasmid_{name}.fastq.gz \
#    03.demux/pacbio_3passes_plasmid_unknown.fastq.gz

In [7]:
# # Number of forward reads
# _3passes_nb_IP_forward_reads=$((`zcat 03.demux/pacbio_3passes_intron_plasmid_forward.fastq.gz | wc -l`/4))
# echo $_3passes_nb_IP_forward_reads
# # Number of reads for Intron Plasmid
# _3passes_nb_IP_reverse_reads=$((`zcat 03.demux/pacbio_3passes_intron_plasmid_reverse.fastq.gz | wc -l`/4))
# echo $_3passes_nb_IP_reverse_reads

### Check for double 5pFS barcode

In [8]:
# # Forward reads
# cutadapt \
#    -b $rev_comp_5p_flanking_seq_barcode \
#    --discard-trimmed \
#    -o 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned.fastq.gz \
#    03.demux/pacbio_3passes_intron_plasmid_forward.fastq.gz

In [9]:
# # Reverse reads
# cutadapt \
#    -b $_5p_flanking_seq_barcode \
#    --discard-trimmed \
#    -o 03.demux/pacbio_3passes_intron_plasmid_reverse_cleaned.fastq.gz \
#    03.demux/pacbio_3passes_intron_plasmid_reverse.fastq.gz

### Combine forward and reverse reads

In [10]:
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned.fastq.gz | wc -l`/4))
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_reverse_cleaned.fastq.gz | wc -l`/4))

In [11]:
# # Reverse complement correct reverse reads
# ## For quality coding: It assumes Illumina (ASCII offset 64), but you have Sanger (offset 33), hence -Q33
# # seqtk has by default has 33 encoding (-Q33)
# module load seqtk/2015-03-25-foss-2014a
# cat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned.fastq.gz \
#    <(seqtk seq -r 03.demux/pacbio_3passes_intron_plasmid_reverse_cleaned.fastq.gz | gzip) \
#    > 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined.fastq.gz

In [12]:
# # Number of combined reads
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined.fastq.gz | wc -l`/4))

### Filter empty vector [NEW - March 2020]

Pour le Intron c’est la sequence au milieu de laquelle l’enhancer est inséré:

w/o enhancer: TGAGGTGTGGCAGGCTTGAGATCTGGCCATACACTTGAGT

In [18]:
# cutadapt multi-threading only supported for -o only 
# https://github.com/marcelm/cutadapt/blob/master/doc/guide.rst
#    --action=none \
cutadapt \
   --action=none \
   -a TGAGGTGTGGCAGGCTTGAGATCTGGCCATACACTTGAGT \
   --output 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors.fastq.gz \
   --untrimmed-output 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_filled_vectors.fastq.gz \
   03.demux/pacbio_3passes_${SAMPLE_NAME}_forward_cleaned_combined.fastq.gz

This is cutadapt 1.18 with Python 3.6.8
Command line parameters: --action=none -a TGAGGTGTGGCAGGCTTGAGATCTGGCCATACACTTGAGT --output 031.demux/pacbio_3passes_intron_plasmid_cleaned_combined_empty_vectors.fastq.gz --untrimmed-output 031.demux/pacbio_3passes_intron_plasmid_cleaned_combined_filled_vectors.fastq.gz 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined.fastq.gz
Processing reads on 1 core in single-end mode ...
Finished in 33.06 s (371 us/read; 0.16 M reads/minute).

=== Summary ===

Total reads processed:                  89,056
Reads with adapters:                     1,098 (1.2%)
Reads written (passing filters):         1,098 (1.2%)

Total basepairs processed:   177,289,751 bp
Total written (filtered):      2,223,432 bp (1.3%)

=== Adapter 1 ===

Sequence: TGAGGTGTGGCAGGCTTGAGATCTGGCCATACACTTGAGT; Type: regular 3'; Length: 40; Trimmed: 1098 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-29 bp: 2; 30-39 bp: 3; 40 bp: 4

Bases preceding removed adapters:

In [19]:
zcat 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_filled_vectors.fastq.gz | grep "^@" | wc -l
zcat 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors.fastq.gz | grep "^@" | wc -l

88463
1108


In [42]:
# Length of the barcode is 18 base pair in Intron plasmid (double-checked with David 26/06/2018)
cutadapt \
   -g CACTCCCAGGTGGCGCGCCCTAG \
   -a TGCAGGTCGCCTGCAGGCTTAAA \
   -n 5 -m18 -M18 --trimmed-only \
   -o 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes.fastq.gz \
   031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors.fastq.gz

This is cutadapt 1.18 with Python 3.6.8
Command line parameters: -g CACTCCCAGGTGGCGCGCCCTAG -a TGCAGGTCGCCTGCAGGCTTAAA -n 5 -m18 -M18 --trimmed-only -o 031.demux/pacbio_3passes_intron_plasmid_cleaned_combined_empty_vectors_barcodes.fastq.gz 031.demux/pacbio_3passes_intron_plasmid_cleaned_combined_empty_vectors.fastq.gz
Processing reads on 1 core in single-end mode ...
Finished in 0.19 s (173 us/read; 0.35 M reads/minute).

=== Summary ===

Total reads processed:                   1,098
Reads with adapters:                     1,098 (100.0%)
Reads that were too short:                  93 (8.5%)
Reads that were too long:                   87 (7.9%)
Reads written (passing filters):           918 (83.6%)

Total basepairs processed:     2,223,432 bp
Total written (filtered):         16,524 bp (0.7%)

=== Adapter 1 ===

Sequence: TGCAGGTCGCCTGCAGGCTTAAA; Type: regular 3'; Length: 23; Trimmed: 1087 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-23 bp: 2

Bases preceding removed ada

In [43]:
zcat 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes.fastq.gz \
   | paste - - - - \
   | cut -f2 \
   > 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes.txt

In [44]:
# Need to remove @
zcat 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes.fastq.gz \
   | paste - - - - \
   | awk '{ print $1,"NEG_CTRL",$2 }' OFS='\t' \
   | sed 's/^.//' \
   > 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes_table.tsv

In [45]:
wc -l 031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes_table.tsv

918 031.demux/pacbio_3passes_intron_plasmid_cleaned_combined_empty_vectors_barcodes_table.tsv


### Separate barcode from enhancers

In [23]:
# _5p_flanking_seq_barcode="CACTCCCAGGTGGCGCGCCCTAG"

In [24]:
# echo $_3p_flanking_seq_barcode

In [25]:
# # Extract barcodes and enhancers based on 5' flanking seq of barcode
# cutadapt \
#    -a $_5p_flanking_seq_barcode --trimmed-only --minimum-length=1 \
#    -o 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_enhancers.fastq.gz \
#    -r 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fastq.gz \
#    03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined.fastq.gz

# # Presence of empty reads if -m option not set (see 5pPlasmid Python notebook)

In [26]:
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_enhancers.fastq.gz | wc -l`/4))
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fastq.gz | wc -l`/4))

In [27]:
# # Adapting the output file of cutadapt to a proper fasta file that can be used to perform in the next cutadapt. 
# zcat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fastq.gz \
#    | sed 's/ /\t/g' | awk '{print ">"$2"\t"$1}' \
#    | sed 's/\t/\n/g' \
#    > 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fasta

In [28]:
# # Reporter BC pairs:
# ##  10 trimmed reads did not generate a reporter-barcode output. this probably means the adapter was at the 3’ end and nothing came after.
# cat 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fasta | sed -n '1~2p'| sort -u | wc -l

### Get barcodes

In [75]:
# Have to trim with both 5' flanking sequence of barcode sequence only because we already trimmed away the 3' flanking sequence
# otherwise it can happen that the barcode is shorter than expected (<17bc) and we would read through the plasmid sequence
# 5' flanking sequence of barcode: GAACACTCGAGCCGAGCAGGCGCGCCGATC
# 3' flanking sequence of barcode: GGACCGGGGCCCGAATTAATTCGCTGTCTG
# At least one read has 2 times the same 5' flanking sequence: >m54094_180531_134158/4588439/ccs. We need to take this into account
# We need to specify -n option because CUTADAPT basic rule with multiple adapter trimming is: "the leftmost match is used for both 5’ and 3’ adapters"
# http://cutadapt.readthedocs.io/en/stable/guide.html#multiple-adapter-occurrences-within-a-single-read

In [29]:
# echo $_5p_flanking_seq_barcode
# echo $_3p_flanking_seq_barcode

In [30]:
# # Length of the barcode is 18 base pair in Intron plasmid (double-checked with David 26/06/2018)
# cutadapt \
#    -a $_3p_flanking_seq_barcode \
#    -n 5 -m18 -M18 --trimmed-only \
#    -o 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta \
#    03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_reporter_and_barcode.fasta

In [31]:
# # How many barcodes
# echo $((`cat 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta | wc -l`/2))
# # How many of the BC are uniq?
# # 35,757 unique Barcodes
# cat 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta | sed -n '2~2p' | sort | uniq | wc -l

In [46]:
# # Filter enhancers based on final barcodes list
# seqtk subseq 03.demux/pacbio_3passes_intron_plasmid_forward_cleaned_combined_clipped_enhancers.fastq.gz \
#    <(grep "^>" 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta \
#    | sed 's/^>//') | gzip \
#    > 03.demux/pacbio_3passes_intron_plasmid_enhancers.fastq.gz

In [47]:
# # How many enhancers
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_enhancers.fastq.gz | wc -l`/4))

## Mapping enhancers

In [48]:
# mkdir 10.mapping

In [49]:
# hg19_fa_path="/ddn1/vol1/staging/leuven/stg_00002/resources/human/hg19/Homo_sapiens_assembly19_sorted.fa"
# hg19_mmi_path="/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/documents/resources/refs/hg19/Homo_sapiens_assembly19_sorted.mmi"

In [50]:
# ls

In [51]:
# # Number of reads
# echo $((`zcat 03.demux/pacbio_3passes_intron_plasmid_enhancers.fastq.gz | wc -l`/4))

In [52]:
# # Map PacBio reads with minimap
# module load Minimap2/2.11-foss-2014a
# module load SAMtools/1.8-foss-2014a
# minimap2 \
#    -ax map-pb \
#    $hg19_mmi_path \
#    03.demux/pacbio_3passes_intron_plasmid_enhancers.fastq.gz \
#    | samtools view -bS > 10.mapping/pacbio_3passes_intron_plasmid_enhancers.bam

In [53]:
# # Sort and create bam index
# samtools sort -o 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.bam 10.mapping/pacbio_3passes_intron_plasmid_enhancers.bam
# samtools index 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.bam

In [54]:
# samtools view -b -q 4 -@ 2 -o 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.bam

In [55]:
# # number of reads mapped after quality filtering
# samtools view 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam | cut -f1 | sort -u | wc -l

In [56]:
# samtools flagstat 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam > 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam.flagstat

In [57]:
# samtools index 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam

In [58]:
# # checking overlap with designed regions (at least 60%)
# module load bedtools/2.27.1-foss-2014a
# intersectBed -f 0.6 \
#    -a 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam \
#    -b amplified_regions.bed -wo -bed > 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed

In [59]:
# # Number of the reads overlapping with designed enhancers
# cat 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed | wc -l

In [60]:
# # Number of unqiue reads overlapping with designed enhancers
# cat 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed | awk '{print $2}' | uniq | wc -l

In [61]:
# # Number of enhancers that are found back in the PacBio -> total of 53 designed enhancers -> 88.4% of the enhancers found back by pack bio
# cat 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed | awk '{print $16}' | uniq | paste -s -d', '
# cat 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed | awk '{print $16}' | uniq | wc -l

## BC-Enhancer Coupling [UPDATE - March 2020]

In [62]:
# # Read ID and barcode sequence in one line 
# # RC the BC
# paste <( cat 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta | sed -n '1~2p'| cut -c 2-) \
#    <( cat 03.demux/pacbio_3passes_intron_plasmid_barcodes.fasta| sed -n '2~2p') \
#    > 20.coupling/pacbio_3passes_intron_plasmid_readID_barcode_table.tsv

In [63]:
# # Make bedfile from bam 
# # Why only keeping 17K regions here?
# bamToBed -i 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bam > 20.coupling/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4.bed

In [64]:
# # Make readID enhancer mapping
# cut -f4,16 10.mapping/pacbio_3passes_intron_plasmid_enhancers.sorted.minQ4_overlapped_with_amplified_regions.bed \
#    > 20.coupling/pacbio_3passes_intron_plasmid_readID_enhancer_table.tsv

In [65]:
# # Combine ReadID-Barcode with ReadID-Enhancer
# awk -v OFS='\t' 'NR==FNR { a[$1]=$0; next } $1 in a { print a[$1], $2 }' \
#    20.coupling/pacbio_3passes_intron_plasmid_readID_enhancer_table.tsv \
#    20.coupling/pacbio_3passes_intron_plasmid_readID_barcode_table.tsv \
#    > 20.coupling/pacbio_3passes_intron_plasmid_readID_enhancer_barcode_table.tsv

In [66]:
group_bc_by_enhancer() {
   awk -F '\t' '                                                      
   $2==x {
      printf ",%s", $3
      next
   }
   {
      x=$2
      printf "\n%s\t%s", x, $3
   }
   END {
      printf "\n"
   }'
}

In [67]:
# [NEW 16 March 2020] Add empty plasmid barcodes
cat \
   20.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_barcode_table.tsv \
   031.demux/pacbio_3passes_${SAMPLE_NAME}_cleaned_combined_empty_vectors_barcodes_table.tsv \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv

In [68]:
cat 20.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_barcode_table.tsv | cut -f3 | sort | uniq | wc -l

33602


In [69]:
cat 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv | cut -f3 | sort | uniq | wc -l

33635


In [70]:
# Group barcodes by enhancer
sort -k2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_list.tsv

In [71]:
# Remove all redundant barcodes
paste \
   <(cut -f1 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_list.tsv) \
   <(cut -f2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_list.tsv | while read line; do echo $line | tr ',' '\n' | uniq | paste -s -d','; done) \
   | sed '/^\s*$/d' > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_unique_barcode_list.tsv

### Barcode quality filtering

#### Q30

In [72]:
# Combine ReadID-Barcode with ReadID-Enhancer
awk \
   -v OFS='\t' 'NR==FNR { a[$1]=$0; next } $1 in a { print a[$1], $2 }' \
   201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv \
   03.demux/pacbio_3passes_${SAMPLE_NAME}_readID_barcode_quality_table_minQ30.tsv \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ30_table.tsv

In [73]:
# Group barcodes by enhancer
sort -k2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ30_table.tsv \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ30_list.tsv

In [74]:
# Remove all redundant barcodes
paste \
   <(cut -f1 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ30_list.tsv) \
   <(cut -f2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ30_list.tsv | while read line; do echo $line | tr ',' '\n' | uniq | paste -s -d','; done) \
   | sed '/^\s*$/d' > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_unique_barcode_minQ30_list.tsv

#### Q60

In [75]:
# Combine ReadID-Barcode with ReadID-Enhancer
awk \
   -v OFS='\t' 'NR==FNR { a[$1]=$0; next } $1 in a { print a[$1], $2 }' \
   201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv \
   03.demux/pacbio_3passes_${SAMPLE_NAME}_readID_barcode_quality_table_minQ60.tsv \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ60_table.tsv

In [76]:
# Group barcodes by enhancer
sort -k2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ60_table.tsv \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ60_list.tsv

In [77]:
# Remove all redundant barcodes
paste \
   <(cut -f1 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ60_list.tsv) \
   <(cut -f2 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_minQ60_list.tsv | while read line; do echo $line | tr ',' '\n' | uniq | paste -s -d','; done) \
   | sed '/^\s*$/d' > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_unique_barcode_minQ60_list.tsv

### Barcode specificity filtering

In [78]:
cat 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_table.tsv \
   | sort -k3 \
   | uniq -f2 \
   | sort -k2 \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_and_negctrl_barcode_final_list.tsv

In [79]:
cat 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ30_table.tsv \
   | sort -k3 \
   | uniq -f2 \
   | sort -k2 \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_enhancer_barcode_and_negctrl_barcode_minQ30_final_list.tsv

In [80]:
cat 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ60_table.tsv \
   | sort -k3 \
   | uniq -f2 \
   | sort -k2 \
   | group_bc_by_enhancer \
   > 201.coupling/pacbio_3passes_${SAMPLE_NAME}_readID_enhancer_and_negctrl_barcode_minQ60_final_list.tsv